# Fault Tolerant Building blocks

Unlike NISQ algorithms which utilize short circuits and many measurements, fault-tolerant algorithms use long circuits and few measurements. The quantum computer can efficiently store large (exponential) systems efficiently but preparing these states is time-consuming and we generally don't need all the information about the state.

## Controlled Operations

A main component of fault-tolerant algorithms is controlled operations. These are defined such that operations given a state of a certain (set of) qubit(s), one can apply an operation to a different set of qubits. For example, the usual CNOT (CX) Gate applies X to the target qubit only if the first qubit is in the $\big|1\big>$ state. This can be written as

$CX\big|0\big>\big|s\big>=\big|0\big>\big|s\big>, \quad CX\big|1\big>\big|s\big>=\big|1\big>X\big|s\big>$

where $s$ is any state $a\big|0\big> + b\big|1\big>$.

Likewise, this can be generalized to multiple controlled operations.

For example, if we want to apply an operation U only when the first 2-qubits are in state 1. We can define


$U^{01}\big|11\big>\big|s\big>=\big|11\big>U\big|s\big>$

In general the operation can be controlled on many qubits and the state $s$ can be many qubits.

In [361]:
from itertools import product
from tangelo.linq import Circuit, Gate, get_backend

# The x-operation is defined as 
sim = get_backend()

cx = Circuit([Gate("CX", 0, control=[1])])

for states in product(["0", "1"], repeat=1):
    f, _ = sim.simulate(Circuit([Gate("X", i+1) for i, state  in enumerate(states) if state == "1"])+cx)
    print(states, f)

('0',) {'00': 1.0}
('1',) {'11': 1.0}


Can you apply the operation controlled on qubits 1 and 2

In [358]:
sim = get_backend()

# They fill in here
ccx = Circuit([Gate("CX", 0, control=[])])

for states in product(["0", "1"], repeat=2):
    f, _ = sim.simulate(Circuit([Gate("X", i+1) for i, state  in enumerate(states) if state == "1"])+ccx)
    print(states, f)

('0', '0') {'000': 1.0}
('0', '1') {'001': 1.0}
('1', '0') {'010': 1.0}
('1', '1') {'111': 1.0}


Now we can combine the operations which will only flip the state if the second and third qubits are in the state "10"

In [360]:

for states in product(["0", "1"], repeat=2):
    f, _ = sim.simulate(Circuit([Gate("X", i+1) for i, state  in enumerate(states) if state == "1"]) + ccx + cx)
    print(states, f)

('0', '0') {'000': 1.0}
('0', '1') {'001': 1.0}
('1', '0') {'110': 1.0}
('1', '1') {'011': 1.0}


In general, we can make more complicated controlled unitaries. One of the key fault-tolerant algorithms is phase-estimation. Which can approximate the eigenvalues of a state using a series of controlled operations follewed by a Quantum Fourier Transform.

## Quantum Phase Estimation

The first fault-tolerant algorithm we will implement in phase-estimation. It is a technique to obtain the energy of an (approximate) eigenstate by a series of controlled time-evolutions of a Hamiltonian. The circuit is shown below

![QPE Circuit](PE_Circuit.png "Quantum Phase Estimation Circuit")

In [2]:
# Define H2 molecule in minimal basis set
from openfermion import get_sparse_operator
import numpy as np

from tangelo import SecondQuantizedMolecule
from tangelo.linq import get_backend, Circuit, Gate
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit

xyz_H2 = [("H", (0., 0., 0.)), ("H", (0., 0., 0.7414))]
mol_H2_sto3g = SecondQuantizedMolecule(xyz_H2, q=0, spin=0, basis="sto-3g")

mol = mol_H2_sto3g

qu_op = fermion_to_qubit_mapping(mol.fermionic_hamiltonian, mapping="JW", n_spinorbitals=mol.n_active_sos, n_electrons=mol.n_active_electrons, spin=mol.spin, up_then_down=False)

# Cheat to get the ground state eigenvalue to be 0.25. The exact ground state energy is 1.137270174660903
qu_op += (0.25 + 1.137270174660903)
eigs, statevectors = np.linalg.eigh(get_sparse_operator(qu_op).toarray())

print(eigs[0])
ground_sv = statevectors[:, 0]
print(eigs[13])
first_sv = statevectors[:, 13]

0.24999999999999947
1.8671062929051834


In [3]:
# Hartree-Fock reference state circuit
hf_circuit = get_reference_circuit(n_spinorbitals=mol.n_active_sos, n_electrons=mol.n_active_electrons, mapping="JW", up_then_down=False, spin=mol.spin)

sim = get_backend("cirq")
f, sv = sim.simulate(hf_circuit, return_statevector=True)

g_ovlp = np.dot(sv, ground_sv)
f_ovlp = np.dot(sv, first_sv)
print(f"The overlap of the initial state with the exact ground state is {g_ovlp}")
print(f"The overlap of the initial state with the exact first excited state is {f_ovlp}")

The overlap of the initial state with the exact ground state is (0.9936146058054713+0j)
The overlap of the initial state with the exact first excited state is (-0.112827368710068+0j)


In [15]:
from tangelo.toolboxes.ansatz_generator.ansatz_utils import trotterize, get_qft_circuit
from tangelo.toolboxes.post_processing.histogram import Histogram


# Reverse order as cirq uses lsq_first
qubit_list = [6, 5, 4]

# State preparation
pe_circuit = hf_circuit + Circuit([Gate("H", q) for q in qubit_list ])

for i, qubit in enumerate(qubit_list):
    # You can play around with how accurate the time-evolution needs to be
    # Use negative time as trotterize uses exp(-iHt) to follow the Schrodinger equation
    pe_circuit += trotterize(qu_op, trotter_order=4, n_trotter_steps=12, time=-2*np.pi*2**i, control=qubit)

# set inverse to true or false
pe_circuit += get_qft_circuit(qubit_list, inverse=True)

sim = get_backend("cirq")
freqs, _ = sim.simulate(pe_circuit)

# Remove qubit indices from histogram corresponding to the state qubits i.e. (0, 1, 2, 3)
hist = Histogram(freqs)
hist.remove_qubit_indices(0, 1, 2, 3)

for key, probability in hist.frequencies.items():
      energy = sum(int(k)/2**(i+1) for i, k in enumerate(key))
      print(f"The probability that the statevector energy={energy} is {probability:3.5f}")

The probability that the statevector energy=0.0 is 0.00004
The probability that the statevector energy=0.125 is 0.00001
The probability that the statevector energy=0.25 is 0.98752
The probability that the statevector energy=0.375 is 0.00005
The probability that the statevector energy=0.5 is 0.00001
The probability that the statevector energy=0.625 is 0.00003
The probability that the statevector energy=0.75 is 0.00005
The probability that the statevector energy=0.875 is 0.01228


Knowing that the energy is 0.25 and the energy is calculated as $E = \sum_{i}^N M[i]1/2^{i}$. We know that the measurement was "010". We can run the circuit agains specifying this is the result we want and then examine the result.

In [16]:
desired_measurement = "010"
pe_plus_measure = pe_circuit + Circuit([Gate("MEASURE", i) for i in qubit_list])

_, sv_new = sim.simulate(pe_plus_measure, desired_meas_result=desired_measurement, return_statevector=True)

print(f"The probability of this measurement is {pe_plus_measure.success_probabilities[desired_measurement]}")

# Shrink vector down to 2**4 size to compare with exact ground state.
sv_new_post = np.reshape(sv_new, (2**4, 2**3))[:, int(desired_measurement, base=2)]

print(f"The final state overlap with the ground state is {np.abs(np.dot(sv_new_post, ground_sv))}")

The probability of this measurement is 0.9875184986388642
The final state overlap with the ground state is 0.9999727387751328


We can see that we measured the desired energy with probability 0.987 and the resulting state has overlap with the exact state of 0.99997 while it started with an overlap of 0.9936.

## Block encodings.

Another building block of many fault-tolerant algorithms is block encodings. This is a technique to implement any operation as long as it can be decomposed into a linear combination of Unitaries. We can implement the operation using $U_{prep}$ and $U_{select}$ defined for a general operator $A=\sum_i c_i U_i$ where $U_i$ are unitaries.

$U_{prep}\big|\psi\big>\big|0\big> = \sum_{i}\sqrt{|c_i|/\alpha}\big|\psi\big>\big|i\big>$

$U_{select}\big|\psi\big>\big|i\big> = U_i\big|\psi\big>\big|i\big>$

where $\alpha$ is the 1-norm of the coefficients $|c_i|$

Applying the circuit $U_{prep}U_{select}U_{prep}^{\dagger}$ results in $A\big|\psi\big>\big|0\big> + \sum_i \big|\psi^{\perp}\big>\big|i\big>$
where $\psi^{\perp}$ are states orthogonal to $A\psi$. This means that if we measure the ancilla qubits and the result is zero, we have successfully applied the desired operation $A$.

For example, let's try to apply $A=\frac{1}{\sqrt{2}}X+\frac{1}{\sqrt{2}}Y$
which is equivalent to
$\left[\begin{array}{cc}0&1/2-1/2\\ 1/2+i/2&0\end{array}\right]$

In [303]:
from tangelo.toolboxes.operators import QubitOperator
from tangelo.linq.helpers.circuits.statevector import StateVector
from tangelo.toolboxes.circuits.lcu import get_uprep_uselect

coefs = [1, 1]

a = QubitOperator("X0", coefs[0]) + QubitOperator("Y0", coefs[1])

vec = np.array(np.abs(coefs))
vec = np.sqrt(vec/np.sum(vec))

# In this case prep can be be applied by simply applying the Hadamard gate to the first qubit
uprep = Circuit([Gate("H", 1)])

sv = StateVector(vec)
uprep = sv.initializing_circuit()
uprep.reindex_qubits([1])


# Uselect
uselect = Circuit([Gate("X", 1)]+[Gate("CX", target=0, control=1)]+[Gate("X", 1)]+[Gate("CY", target=0, control=1)])

sim = get_backend("cirq")

state_prep = Circuit([Gate("H", 0)])


#uprep, uselect, _, _, alpha = get_uprep_uselect(a, make_alpha_eq_2=False)
#print(alpha)

block_circuit = uprep + uselect + uprep.inverse() 

circuit = state_prep + block_circuit + Circuit([Gate("MEASURE", 1)])
f, sv = sim.simulate(circuit, desired_meas_result="0", return_statevector=True)
final_state = sv.reshape(2,2)[:,0]
print(final_state)
print(circuit.success_probabilities)

[0.5-0.5j 0.5+0.5j]
{'0': 0.5000000000000001}


In [333]:
final_exact = get_sparse_operator(a).toarray()@np.array([1/np.sqrt(2), 1/np.sqrt(2)])
final_exact /=np.linalg.norm(final_exact)
print(np.vdot(final_state, final_exact))

(1+0j)


## Amplitude Amplification

We would like to have a higher probability of success. To implement this we need to apply signed operations.

$F_{good} F_{full}$

where $F_{good} = (1-\big|0\big>^{ancilla}\big<0\big|^{ancilla})$ and  $F_{full} = $

In [334]:
from tangelo.toolboxes.circuits.lcu import sign_flip
# Flip sign on good state
sfa = Circuit([Gate("X", 1), Gate("PHASE", 1, parameter=np.pi), Gate("X", 1)])
sft = sign_flip([0, 1])
print(sfa)

circuit = state_prep + block_circuit

# 1 application of amplitude amplification
amplifying_circuit = (sfa  + circuit.inverse() + sft + circuit)

sim.simulate(state_prep + block_circuit + amplifying_circuit, return_statevector=True)

Circuit object. Size 3 

X         target : [1]   
PHASE     target : [1]   parameter : 3.141592653589793
X         target : [1]   



({'00': 0.2500000000000001, '01': 0.25, '10': 0.2500000000000001, '11': 0.25},
 array([-0.35355339+0.35355339j, -0.35355339-0.35355339j,
        -0.35355339-0.35355339j, -0.35355339+0.35355339j]))